# Finite Difference Option Pricing

## Finite Difference Methods

Finite Difference Methods (FDM) are one of the popular numerical methods used in computational finance. FDM are discretizations used for solving differential equations by approximating them with difference equations. It is one of the simplest and the oldest methods to solve differential equations. These techniques were applied to numerical applications as early as 1950s.

FDM are similar in approach to the (binomial) tress. However, instead of discretizing asset prices and the passage of time in a tree structure, it discretizes in a grid - with time and price steps - by calculating the value at every possible grid points.

Explicit, Implicit and Crank-Nicolson are the three popular approaches of FDM. The explicit methods are simple to implement, but it does not always converge and largely depends on the size of the time and asset step. Explicit methods are unstable when compared to other two methods. Finite Difference approach is peferred for low dimensional problem, usually upto 4 dimensions.

## Differentiation Using The Grid

The Binomial method contains the diffusion - the volatility - in the tree structure whereas in FDM, the 'tree' is fixed and we change the parameters to reflect the changing diffusion. We will now define the grid by specifying the time step $\delta t$ and asset step $\delta s$ and discretize $S$ and $t$ as

$$
S=i \delta s
$$

and time to maturity as

$$
t=T-k \delta t
$$

where $0 \leq i \leq I$ and $0 \leq k \leq K$

Here $i$ and $k$ are respective steps in the grid and we can write the value of the option at each grid points as

$$
V_{i}^{k}=(i \delta S, T-k \delta t)
$$

## Approximating Greeks

The greeks terms, the Black-Scholes equation can be written as

$$
\Theta+\frac{1}{2} \sigma^{2} S^{2} \Gamma+r S \Delta-r V=0
$$

Assume that we know the option value at each grid points, we can extract the derivatives of the option using Taylor series expansion.

### Approximating $\Theta$

We know that the first derivative of option as,

$$
\frac{\partial V}{\partial t}=\lim _{h \rightarrow 0} \frac{V(S, t+h)-V(S, t)}{h}
$$

We can then approximate the time derivative from our grid using

$$
\frac{\partial V}{\partial t}(S, t) \approx \frac{V_{i}^{k}-V_{i}^{k+1}}{\delta t}
$$

### Approximating $\Delta$

From the lecture, we know that the central difference has much lower error when compared to forward and backward differences. Accordingly, we can approximate the first derivative of option with respect to the underlying as

$$
\frac{\partial V}{\partial S}(S, t) \approx \frac{V_{i+1}^{k}-V_{i-1}^{k}}{2 \delta S}
$$

### Approximating $\Gamma$

The gamma of the option is the second derivative of option with respective to the underlying and approximating it we have,

$$
\frac{\partial V^{2}}{\partial S^{2}}(S, t) \approx \frac{V_{i+1}^{k}-2 V_{i}^{k}+V_{i-1}^{k}}{\delta S^{2}}
$$

#### Import Required Libraries

In [1]:
# Importing libraries
import pandas as pd
from numpy import *

# plotting
import cufflinks as cf
cf.set_config_file(offline=True)

# rendering
# import plotly.io as pio
# pio.renderers.default = 'notebook_connected'
# Set max row and columns to 50
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)

#### Example

Suppose that we know the value of the option on the below grid points, we can then easily evaluate the greeks as follows

![](https://cdn.mathpix.com/cropped/2023_08_25_8e3a8c54d4fcc061b7c3g-03.jpg?height=1252&width=1358&top_left_y=740&top_left_x=275)

From the grid, we can estimate the

$$
\begin{gathered}
\Theta=\frac{12-13}{0.1}=-10 \\
\Delta=\frac{15-10}{2 x 2}=1.25 \\
\Gamma=\frac{15-2 x 12+10}{2 x 2}=0.25
\end{gathered}
$$

Black Scholes Equation is a relationship between the option value and greeks. If we know the option value at the expiration, we can step back to get the values prior to it such that $V_{k}^{i}=V_{k-1}^{i}-\Theta * d t$. This approach is called Explicit Finite Difference Method because the relationship between the option values at time step $k$ is a simple function of the option values at time step $k-1$.

## Option Pricing Techniques

As with other option pricing techniques Explicit Finite Difference methods are used to price options using what is essentially a three step process.

**Step 1:** Generate the grid by specifying grid points.

**Step 2:** Specify the final or initial conditions.

**Step 3:** Use boundary conditions to calculate option values and step back down the grid to fill it.

#### European Option

To price an option, we generate a finite grid of a specified asset and time steps for a given maturity. Next, we specify the initial and boundary conditions to calculate payoff when $\mathrm{S}$ and $\mathrm{T}$ equals zero. We then step back to fill the grid with newer values derived from the earlier values.

#### Specify Parameters

In [2]:
# Specify the parameters for FDM

T = 1 # time to maturity in years
E = 100 # strike price
r = .05 # riskfree rate
vol = .20 # volatility
Flag = 1 # Flag = 1 for call, -1 for puts
NAS = 20 # number of asset steps
ds = 2* E / NAS # asset step size
dt = (0.9/vol**2/NAS**2) # time step size, for stability
NTS = int(T / dt) + 1 # number of time steps
dt = T / NTS # time step size [Expiration as int # of time steps away]

### Generate Grid

Build the grid with the above parameters

In [4]:
# Create asset steps i*ds
s = arange(0, (NAS+1)*ds,ds)
s

array([  0.,  10.,  20.,  30.,  40.,  50.,  60.,  70.,  80.,  90., 100.,
       110., 120., 130., 140., 150., 160., 170., 180., 190., 200.])

In [5]:
# Create time steps k*dt
t = T-arange(NTS*dt,-dt,-dt)
t

array([0.        , 0.05555556, 0.11111111, 0.16666667, 0.22222222,
       0.27777778, 0.33333333, 0.38888889, 0.44444444, 0.5       ,
       0.55555556, 0.61111111, 0.66666667, 0.72222222, 0.77777778,
       0.83333333, 0.88888889, 0.94444444, 1.        ])

In [6]:
# Verify the steps size
s.shape, t.shape

((21,), (19,))

In [7]:
#Initialize the grid with zeros
grid = zeros((len(s),len(t)))

# Subsume the grid points into a dataframe
# with asset price as index and time steps as columns
grid = pd.DataFrame(grid, index=s, columns=around(t,3))
grid

,0.000,0.056,0.111,0.167,0.222,0.278,0.333,0.389,0.444,0.500,0.556,0.611,0.667,0.722,0.778,0.833,0.889,0.944,1.000
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
60.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
70.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
80.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Specify Condition

Specify the final condition for the option and the payoff
$$
V_{i}^{0}=\max (i \delta s-E, 0)
$$

In [8]:
# Set Final or Initial condition at Expiration
if Flag == 1:
    grid.iloc[:,0] = maximum(s - E, 0)
else:
    grid.iloc[:,0] = maximum(E - s, 0)

In [9]:
# Verify the grid
grid

,0.000,0.056,0.111,0.167,0.222,0.278,0.333,0.389,0.444,0.500,0.556,0.611,0.667,0.722,0.778,0.833,0.889,0.944,1.000
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
60.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
70.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
80.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Fill the Grid

Specify boundary condition at $\mathrm{S}=0$

$$
V_{0}^{k}=(1-r \delta t) V_{0}^{k-1}
$$

Specify boundary condition at $\mathrm{S}=\infty$

$$
V_{i}^{k}=2 V_{i-1}^{k}-V_{i-2}^{k}
$$

In [10]:
# k is counter
for k in range(1, len(t)):
    for i in range(1,len(s)-1):
        delta = (grid.iloc[i+1,k-1] - grid.iloc[i-1,k-1]) / (2*ds)
        gamma = (grid.iloc[i+1,k-1]-2*grid.iloc[i,k-1]+grid.iloc[i-1,k-1]) / (ds**2)
        theta = (-0.5* vol**2 * s[i]**2 * gamma) - (r*s[i]*delta) + (r*grid.iloc[i,k-1])
        
        grid.iloc[i,k] = grid.iloc[i,k-1] - (theta*dt)

        # Set boundary condition at S = 0
        grid.iloc[0,k] = grid.iloc[0,k-1] * (1-r*dt) # ds = rsdt + sigma*sdx, s= 0, ds = 0

        # Set boundary condition at S = infinity # gamma = 0, so you can linearly extract
        grid.iloc[len(s)-1,k] = 2*grid.iloc[len(s)-2,k] - grid.iloc[len(s)-3,k]

        # Round grid values to 2 decimals
        grid = around(grid,3)

# Verify the grid
grid

,0.000,0.056,0.111,0.167,0.222,0.278,0.333,0.389,0.444,0.500,0.556,0.611,0.667,0.722,0.778,0.833,0.889,0.944,1.000
0.0,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
10.0,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
20.0,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
30.0,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
40.0,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
50.0,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.002,0.003,0.004,0.006
60.0,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.002,0.004,0.007,0.011,0.016,0.023,0.031,0.041,0.053,0.066
70.0,0.0,0.000,0.000,0.000,0.001,0.003,0.008,0.016,0.028,0.045,0.067,0.094,0.126,0.164,0.207,0.255,0.308,0.366,0.429
80.0,0.0,0.000,0.000,0.011,0.037,0.080,0.141,0.218,0.310,0.416,0.534,0.662,0.799,0.944,1.096,1.254,1.417,1.584,1.755
90.0,0.0,0.000,0.128,0.336,0.592,0.878,1.181,1.494,1.812,2.131,2.450,2.767,3.081,3.392,3.700,4.005,4.306,4.604,4.899


## Visualise the payoff

In [12]:
# Plot Call Option Payoff
grid.iloc[0:15,:].iplot(kind = 'surface', title='Call Option values by Explicit FDM', legend=True)

## Bilinear Interpolation

We have generated the grid and filled it with the possible option values. However, if we have to estimate option value or its derivatives on the mesh points, how can we estimate the value at points in between? The simplest way is to do a two-dimensional interpolation method called Bilinear Interpolation.